In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 1. Installazione libreria
!pip install bing-image-downloader

from bing_image_downloader import downloader
import os
import shutil
import signal

# 2. Auto da riconoscere
ricerche = {
    """"Fiat_Panda": [
        #"Fiat Panda 2018",
        "Fiat Panda 2020",
        "Fiat Panda 3 serie",
        "Fiat Panda usata",
    ],
    "Fiat_500": [
        #"Fiat 500 usata 2018 strada",
        #"Fiat 500 parcheggiata città",
        "Fiat 500 2017 usata"
    ],"""
    "Tesla_Model_3": [
        "Tesla Model 3 usata",
        #"Tesla Model 3 charging street",
        #"Tesla Model 3 dirty"
    ],
    "Dacia_Duster": [
        #"Dacia Duster 2019 offroad",
        "Dacia Duster usata",
        #"Dacia Duster 2020 strada"
    ]
}

# 3. Scarica le immagini automaticamente
for cartella, keywords in ricerche.items():
    for key in keywords:
        print(f"Scaricando: {key}...")
        downloader.download(
            key,
            limit=30,
            output_dir="/content/drive/MyDrive/PM_project/dataset_auto",
            adult_filter_off=True,
            force_replace=False,
            timeout=10,
            verbose=True
        )
        print(f"{key} scaricata")

In [ ]:
import os
import uuid
from PIL import Image

# Configurazione
dataset_path = "/content/drive/MyDrive/PM_project/dataset_auto"

print(f"--- Inizio pulizia e rinomina in {dataset_path} ---")
deleted_count = 0
renamed_count = 0

# Scansiona tutte le sottocartelle
for folder in os.listdir(dataset_path):
    folder_path = os.path.join(dataset_path, folder)

    if os.path.isdir(folder_path):
        print(f"\nElaborazione cartella: {folder}")

        valid_temp_files = []

        # --- FASE 1: CONTROLLO E RINOMINA TEMPORANEA ---
        for filename in os.listdir(folder_path):
            file_path = os.path.join(folder_path, filename)

            if os.path.isdir(file_path):
                continue

            try:
                with Image.open(file_path) as img:
                    img.load() 
                    print(f"Analisi file {filename}")

                    # Controllo formato
                    if img.format not in ['JPEG', 'PNG', 'GIF', 'BMP']:
                        print(f"Formato errato ({img.format}): {filename} -> ELIMINATO")
                        os.remove(file_path)
                        deleted_count += 1
                    else:
                        # Se è valida: rinomina temporanea per evitare conflitti
                        # Usato UUID per avere un nome univoco sicuro
                        ext = os.path.splitext(filename)[1].lower()
                        if not ext: # Se non ha estensione, viene dedotta dal formato
                            ext = f".{img.format.lower()}"
                            if ext == ".jpeg": ext = ".jpg"

                        temp_name = f"temp_{uuid.uuid4()}{ext}"
                        temp_path = os.path.join(folder_path, temp_name)

                        os.rename(file_path, temp_path)
                        valid_temp_files.append(temp_path)

            except Exception as e:
                print(f"File corrotto: {filename} -> ELIMINATO")
                if os.path.exists(file_path):
                    os.remove(file_path)
                    deleted_count += 1

        # --- FASE 2: RINOMINA FINALE INCREMENTALE (Image_1, Image_2...) ---
        valid_temp_files.sort()
        total_files = len(valid_temp_files)

        for i, temp_path in enumerate(valid_temp_files, 1):
            # Recupera estensione dal file temporaneo
            print(f"Riordinamento file {i} su {total_files}")
            ext = os.path.splitext(temp_path)[1]

            # Crea il nuovo nome finale: Image_1.jpg, Image_2.jpg...
            final_name = f"Image_{i}{ext}"
            final_path = os.path.join(folder_path, final_name)

            os.rename(temp_path, final_path)
            renamed_count += 1

        print(f"Rinominate {len(valid_temp_files)} immagini in sequenza.")

# Rimozionr checkpoint di Colab
checkpoint_path = os.path.join(dataset_path, '.ipynb_checkpoints')
if os.path.exists(checkpoint_path):
    import shutil
    shutil.rmtree(checkpoint_path)

print(f"\nOPERAZIONE COMPLETATA!")
print(f"File eliminati: {deleted_count}")
print(f"File rinominati e ordinati: {renamed_count}")

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import os
import numpy as np

# --- CONFIGURAZIONE ---
DATASET_DIR = "/content/drive/MyDrive/PM_project/dataset_auto"
IMG_SIZE = 224      
BATCH_SIZE = 32
EPOCHS = 20         
NUM_CLASSES = 4

print(f"Analizzando le GPU disponibili...")
if not tf.config.list_physical_devices('GPU'):
    print("ATTENZIONE: Nessuna GPU rilevata")
else:
    print("GPU Rilevata! Il training sarà veloce.")

# --- 1. CARICAMENTO DATI ---
print("\n--- Caricamento Dataset ---")
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    crop_to_aspect_ratio=True
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATASET_DIR,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    crop_to_aspect_ratio=True
)

class_names = train_ds.class_names
print(f"Classi trovate: {class_names}")

# Pre-processing per MobileNetV2 (porta i valori da 0-255 a -1 a 1)
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input

train_ds = train_ds.map(lambda x, y: (preprocess_input(x), y))
val_ds = val_ds.map(lambda x, y: (preprocess_input(x), y))

# Ottimizzazione della cache per velocizzare il caricamento
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# --- 2. DATA AUGMENTATION ---
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal"),
  layers.RandomRotation(0.1),
  layers.RandomContrast(0.2),
  layers.RandomTranslation(0.1, 0.1)
])

# --- 3. COSTRUZIONE MODELLO (MobileNetV2) ---
print("\n--- Costruzione Modello ---")

# Scaricamento modello pre-addestrato (MobileNetV2)
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,
    weights='imagenet'
)

# Congelamento della base
base_model.trainable = False

inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = data_augmentation(inputs) 
x = base_model(x, training=False)
x = layers.GlobalAveragePooling2D()(x) 
x = layers.Dropout(0.2)(x)

# STRATO FINALE:
outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = models.Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# --- 4. TRAINING ---
print("\n--- Inizio Training... ---")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS
)

# --- 5. GRAFICI ---

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Accuracy')

plt.subplot(1, 2, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Loss')
plt.show()

# --- 6. ESPORTAZIONE PER ANDROID ---
print("\n--- Conversione in TFLite per Android ---")

# Conversione del modello
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Salvataggio del file .tflite
with open('/content/drive/MyDrive/PM_project/car_recognizer.tflite', 'wb') as f:
    f.write(tflite_model)

# Salvataggio delle etichette (labels.txt)
with open('/content/drive/MyDrive/PM_project/labels.txt', 'w') as f:
    for name in class_names:
        f.write(name + '\n')


In [ ]:
# --- FASE 2: FINE TUNING ---
print("\n--- Inizio Fine Tuning ---")

# 1. Scongelamento del cervello di MobileNet
base_model.trainable = True

# Scongelati solo gli ultimi 50 strati (quelli che vedono i dettagli complessi)
# lasciando congelati i primi 100 (che vedono linee e cerchi base).
print(f"Numero totale strati nella base: {len(base_model.layers)}")

fine_tune_at = 100

# Congela tutti i layer prima del numero 100
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

# 2. Ricompiliamo con un Learning Rate molto basso
model.compile(loss='sparse_categorical_crossentropy',
              optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5),
              metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=15,               
    restore_best_weights=True,
    verbose=1
)

model.summary()

# 3. Addestriamo per altri 10 giri (Epoche)
total_epochs = 40

print(f"\n--- Ripartiamo da dove eravamo rimasti per altri 20 giri ---")

history_fine = model.fit(train_ds,
                         epochs=total_epochs,
                         initial_epoch=history.epoch[-1],
                         validation_data=val_ds,
                         callbacks=[early_stopping]
                         )

# --- 4. NUOVI GRAFICI E SALVATAGGIO ---

acc += history_fine.history['accuracy']
val_acc += history_fine.history['val_accuracy']
loss += history_fine.history['loss']
val_loss += history_fine.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.plot([EPOCHS-1,EPOCHS-1], plt.ylim(), label='Inizio Fine Tuning')
plt.legend(loc='lower right')
plt.title('Accuracy dopo Fine Tuning')
plt.show()

# Modello fine tuned
print("\n--- Salvataggio Modello nuovo ---")
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open('/content/drive/MyDrive/PM_project/car_recognizer_tuned.tflite', 'wb') as f:
    f.write(tflite_model)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Prendiamo tutte le immagini di validazione e le loro etichette reali
print("Calcolo delle predizioni in corso...")
y_true = []
y_pred = []

for images, labels in val_ds:

    # Eseguiamo la predizione
    preds = model.predict(images, verbose=0)
    # Convertiamo le probabilità in numeri interi (l'indice della classe vincente)
    preds_classes = np.argmax(preds, axis=1)

    y_true.extend(labels.numpy())
    y_pred.extend(preds_classes)

# 2. Generiamo il Report Testuale (Precision, Recall, F1-Score)

print("\n--- PAGELLA PER CLASSE ---")
print(classification_report(y_true, y_pred, target_names=class_names))

# 3. Disegniamo la Matrice di Confusione (Grafico)
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Cosa ha predetto il Modello')
plt.ylabel('Cosa era REALMENTE')
plt.title('Confusion Matrix')
plt.show()